In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
import numpy as np
import os
import glob

# Get all CSV files in the current directory
csv_files = glob.glob("*.csv")

# Loop through each CSV file
for file_path in csv_files:
    file_prefix = os.path.splitext(file_path)[0]  # Extract the file name without extension
    data = pd.read_csv(file_path, usecols=['x', 'y', 'avg_traversal_steps', 'avg_intersection_tests'], 
                       dtype={'x': 'int32', 'y': 'int32', 'avg_traversal_steps': 'float32', 'avg_intersection_tests': 'float32'})

    # Group data
    grouped_data = data.groupby(['y', 'x'], as_index=False).mean()

    # Create heatmap 
    max_y = grouped_data['y'].max() + 1  # Indices start from 0
    max_x = grouped_data['x'].max() + 1

    # Initialize grids 
    heatmap_data_1 = pd.DataFrame(0, index=range(max_y), columns=range(max_x), dtype='float32')
    heatmap_data_2 = pd.DataFrame(0, index=range(max_y), columns=range(max_x), dtype='float32')

    # Fill the heatmaps
    for _, row in grouped_data.iterrows():
        heatmap_data_1.at[row['y'], row['x']] = row['avg_traversal_steps']
        heatmap_data_2.at[row['y'], row['x']] = row['avg_intersection_tests']

    # Convert the data to NumPy arrays for faster plotting 
    heatmap_data_1 = heatmap_data_1.values
    heatmap_data_2 = heatmap_data_2.values

    # Measure statistics for heatmap data
    stats = {
        'avg_traversal_steps': {
            'mean': np.nanmean(heatmap_data_1),
            'min': np.nanmin(heatmap_data_1),
            'max': np.nanmax(heatmap_data_1)
        },
        'avg_intersection_tests': {
            'mean': np.nanmean(heatmap_data_2),
            'min': np.nanmin(heatmap_data_2),
            'max': np.nanmax(heatmap_data_2)
        }
    }

    # Save statistics to a text file
    stats_file_path = f"{file_prefix}_heatmap_statistics.txt"
    with open(stats_file_path, 'w') as f:
        for key, values in stats.items():
            f.write(f"{key} Statistics:\n")
            for stat, value in values.items():
                f.write(f"  {stat}: {value}\n")
            f.write("\n")

    # Create a custom colormap (green -> yellow -> red)
    custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', ['green', 'yellow', 'red'])

    # Normalization 
    norm1 = Normalize(vmin=heatmap_data_1.min(), vmax=heatmap_data_1.max())
    norm2 = Normalize(vmin=heatmap_data_2.min(), vmax=heatmap_data_2.max())

    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))

    # Heatmap 1
    im1 = axes[0].imshow(heatmap_data_1, cmap=custom_cmap, norm=norm1, aspect='auto')
    axes[0].set_title(f'{file_prefix}: Heatmap of avg_traversal_steps')
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')

    # Heatmap 1 colorbar
    cbar1 = fig.colorbar(im1, ax=axes[0], orientation='vertical')
    cbar1.set_label('avg_traversal_steps')

    # Heatmap 2 
    im2 = axes[1].imshow(heatmap_data_2, cmap=custom_cmap, norm=norm2, aspect='auto')
    axes[1].set_title(f'{file_prefix}: Heatmap of avg_intersection_tests')
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')

    # Heatmap 2 colorbar
    cbar2 = fig.colorbar(im2, ax=axes[1], orientation='vertical')
    cbar2.set_label('avg_intersection_tests')

    plt.tight_layout()

    # Save the heatmaps to disk
    heatmap_image_path = f"{file_prefix}_heatmap_visualization.png"
    plt.savefig(heatmap_image_path)
    plt.close()

    print(f"Processed and saved results for {file_path}")


Processed and saved results for s3 grid.csv
Processed and saved results for s3 mid.csv
Processed and saved results for s3 oct.csv
Processed and saved results for s3 sah .csv
